In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_feats_0.2.csv')
#df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
#df =pd.read_csv('../../data/more_features/more_feats_new_labels_0.1_noisy_0.6.csv')
df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_2_missing_05.csv')
#df = utils.balance_dataset(df, 8000)
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,6.863691,341.823457,-1.000000,0.681783,284.835163,92.819484,-1.000000,2.218400,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,20.591074,-1.000000,Inconclusive diagnosis
1,8.745950,104.470248,0.461482,0.787183,409.993509,104.531457,77.222298,2.510043,1,0.713849,-1.000000,51.527756,7.722555,7.822857,-1.0,26.237850,18.835005,Vitamin B12/Folate deficiency anemia
2,10.308881,475.936322,-1.000000,6.155778,-1.000000,104.647240,95.040788,2.955323,1,-1.000000,18.285577,100.169515,54.471371,11.239513,-1.0,30.926642,-1.000000,Vitamin B12/Folate deficiency anemia
3,7.525442,-1.000000,1.972946,2.172161,-1.000000,97.271565,-1.000000,2.320958,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,22.576325,-1.000000,No anemia
4,9.544870,411.496642,0.891182,0.000000,236.428214,104.721025,-1.000000,2.734371,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,28.634611,-1.000000,Unspecified anemia


In [4]:
df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [5]:
utils.get_dt_performance(df)

(0.7139285714285715,
 0.7119085995948692,
 0.833481105802754,
 datetime.timedelta(microseconds=4001))

In [6]:
df.label.value_counts()

No anemia                               16000
Anemia of chronic disease                8803
Iron deficiency anemia                   8304
Aplastic anemia                          8158
Unspecified anemia                       8106
Hemolytic anemia                         8075
Vitamin B12/Folate deficiency anemia     8074
Inconclusive diagnosis                   4480
Name: label, dtype: int64

In [7]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [8]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [9]:
len(action_list)

25

In [10]:
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,6.863691,341.823457,-1.000000,0.681783,284.835163,92.819484,-1.000000,2.218400,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,20.591074,-1.000000,7
1,8.745950,104.470248,0.461482,0.787183,409.993509,104.531457,77.222298,2.510043,1,0.713849,-1.000000,51.527756,7.722555,7.822857,-1.0,26.237850,18.835005,1
2,10.308881,475.936322,-1.000000,6.155778,-1.000000,104.647240,95.040788,2.955323,1,-1.000000,18.285577,100.169515,54.471371,11.239513,-1.0,30.926642,-1.000000,1
3,7.525442,-1.000000,1.972946,2.172161,-1.000000,97.271565,-1.000000,2.320958,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,22.576325,-1.000000,0
4,9.544870,411.496642,0.891182,0.000000,236.428214,104.721025,-1.000000,2.734371,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,28.634611,-1.000000,2


#### Testing

In [51]:
training_env = utils.create_env(X_train, y_train)
dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_2_missing_05_8000000', 
                            training_env)
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
test_df.head()

Using stable baselines 3
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,4.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, Iron deficiency anemia]",4.0,4.0
1,4.0,1.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, Anemia of chronic dise...",3.0,3.0
2,4.0,2.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, Anemia of chronic dise...",3.0,3.0
3,5.0,3.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, tibc, Iron deficienc...",4.0,4.0
4,4.0,4.0,0.0,-1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",2.0,5.0


In [52]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

73.65

In [53]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(3.9054285714285712, 0.347)

In [54]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.7365, 0.7453048708164235, 0.8667508271174059)

In [55]:
test_df.y_pred.unique()

array([4., 3., 5., 1., 0., 7., 6., 2.])

In [32]:
test_df[test_df.y_pred==4]

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,5.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, hematocrit, Iron defic...",4.0,4.0
3,5.0,3.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, tibc, Iron deficienc...",4.0,4.0
8,5.0,8.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, hematocrit, Iron defic...",4.0,4.0
11,6.0,11.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, hematocrit, gender, Ir...",4.0,4.0
49,6.0,49.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, hematocrit, gender, Ir...",4.0,4.0
...,...,...,...,...,...,...,...,...
13951,5.0,13951.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, tibc, Iron deficienc...",4.0,4.0
13954,6.0,13954.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, hematocrit, gender, Ir...",4.0,4.0
13976,5.0,13976.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, hematocrit, Iron defic...",4.0,4.0
13983,5.0,13983.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, tibc, Iron deficienc...",4.0,4.0


#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/test_df3_5500000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/success_df3_5500000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
# test_df = pd.read_csv('../../test_dfs/many_features/0.1/correlated/test_df3_missing3_12500000.csv')
# test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])